In [53]:
library(data.table)
library(TSrepr)
library(TSdist)
library(dtw)
library(TunePareto)
library(dplyr)

# Summary

In this homework, 60 different parameter combination will be tried to get hyperparameter tuning. To get this combination, 5 different representation types(raw + 2 different difference taken + 2 different PAA), 4 different distance calculations, and 3 different k values will be used. Difference will be taken by shift function and 1, 2 will be used to get difference values. For PAA, 2 different segment length will be used by considering proper values for timeseries. In terms of distance, Euclidean, Dynamic Time Warping, Longest Common Subsequence, and Edit Distance with Real Penalties will be used. Lastly, k values are selected as 1, 3, 5 for k-nearest neighbor model. 

Performance of the models will be inspected by using 10-fold cross and 5 repeated validation technique. Moreover, performance of the models will be controlled by having the same test indices to have identical conditions. Lastly, obtained 60 performance will be controlled by a dataframe including average accuracy and standard deviation in accuracy. By considering this dataframe, the best parameter combination will be controlled and this parameters will be used to get accuracy performance in the test dataset. Last comments will be held at the end of the notebooks.

### Parameter Selections

- Representation Type(5): Raw, 1 Difference Taken, 2 Difference Taken, Piecewise Aggregate Approximation(PAA) with 4 segment length, Piecewise Aggregate Approximation(PAA) with 8 segment length
- Distance Calculation(4): Euclidean, Dynamic Time Warping with default parameters, Longest Common Subsequence(LCSS) with epsilon 0.05, Edit Distance with Real Penalties(ERP) with epsilon 0.5
- k values of nearest neighbor model(3): 1, 3, 5

# Context
(In order to get specified techniques rapidly, a context part is added to Notebook.)

1. [Data Preparation](#1)
1. [Classify Function](#2)
1. [Representations](#3)
    1. Raw Data
    1. [Difference Function](#4)
    1. [PAA Function](#5)
1. [Distances](#6)
    1. [Raw Data](#7)
    1. [Difference Data](#8)
    1. [PAA Data](#9)
1. [Main Model](#10)
1. [Result of Models](#11)
1. [Test Performance](#12)
1. [Comments](#13)

<a id="1"></a>
# Data Preparation

In [2]:
dataset_path="D:/Datasets/Univariate2018_arff/Univariate_arff/"

In [3]:
distance_path="C:/Users/bahad/GitHub/IE48B/Homework3/Distances/"

In [4]:
first_dataset="Beef"
second_dataset="BirdChicken"
third_dataset="BME"
fourth_dataset="Coffee"
fifth_dataset="Wine"

## Loading Dataset

In [5]:
traindata=as.matrix(fread(sprintf('%s%s/%s_TRAIN.txt',dataset_path, third_dataset,third_dataset)))

In [6]:
head(traindata)

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129
1,0.0377646330,0.0002531124,0.003977076,0.02076406,0.02173118,0.114000000,0.41400000,0.71800000,0.88900000,...,0.034208601,0.02283097,0.013387386,0.038906948,0.039187692,0.0536167050,0.054673016,0.031523157,0.03978737,0.02066080
1,0.0004248819,0.0209818240,0.002525833,0.02011753,0.03707394,0.034782011,0.08300000,0.31700000,0.55000000,...,0.036964033,0.01122764,0.025040340,0.044364509,0.043476064,0.0441632130,0.021766566,0.007789026,0.02919420,0.02183739
1,0.0198557600,0.0036765858,0.035077439,0.01154532,0.12700000,0.462000000,0.79700000,0.98500000,1.07400000,...,0.043864909,0.01768577,0.046659284,0.005716803,0.019495807,0.0215920930,0.018679618,0.022478860,0.03220358,0.04518721
1,-0.0058045195,0.0026662398,0.021715744,0.01695835,0.02703698,0.003595671,0.01259742,0.09000000,0.33800000,...,0.007400456,0.02147181,0.005443165,0.045583447,0.001083856,0.0190912200,0.005705397,0.003578330,0.03154213,0.03438603
1,0.0313804330,0.0570924850,0.047269203,0.04387237,0.02690178,0.037878251,0.03226138,0.01146470,0.02436203,...,0.035658042,0.01919065,0.044963907,0.014297094,0.005419482,-0.0002304543,0.027151247,0.029029138,0.04078212,0.03472307
1,0.0269493680,0.0465532830,0.012495950,0.04275020,0.02778757,0.038858944,0.01938303,0.01827375,0.12000000,...,0.014155193,0.01372328,0.046840932,0.041271348,0.051914020,0.0581532340,0.039676077,0.042518082,0.03827971,0.03426204


In [7]:
str(traindata)

 num [1:30, 1:129] 1 1 1 1 1 1 1 1 1 1 ...
 - attr(*, "dimnames")=List of 2
  ..$ : NULL
  ..$ : chr [1:129] "V1" "V2" "V3" "V4" ...


## Class Information

In [8]:
trainclass=traindata[,1] 

## Time Series

In [9]:
traindata=traindata[,2:ncol(traindata)]

## Dataset Information

In [10]:
tlength=ncol(traindata)
n_series_train=nrow(traindata)

## Indices for Datasets

Mentioned test indices are obtained by TunePareto library to have identical conditions. nfold and ntimes parameters are selected as 10, 5 respectively.

In [11]:
set.seed(35)
nof_rep=5
n_fold=10

In [12]:
cv_indices=generateCVRuns(trainclass, ntimes =nof_rep, nfold = n_fold, 
                          leaveOneOut = FALSE, stratified = TRUE)

str(cv_indices)

List of 5
 $ Run  1:List of 10
  ..$ Fold  1 : int [1:3] 10 19 27
  ..$ Fold  2 : int [1:3] 6 20 23
  ..$ Fold  3 : int [1:3] 8 11 30
  ..$ Fold  4 : int [1:3] 1 13 26
  ..$ Fold  5 : int [1:3] 3 15 25
  ..$ Fold  6 : int [1:3] 7 18 24
  ..$ Fold  7 : int [1:3] 2 17 22
  ..$ Fold  8 : int [1:3] 5 12 21
  ..$ Fold  9 : int [1:3] 4 16 29
  ..$ Fold  10: int [1:3] 9 14 28
 $ Run  2:List of 10
  ..$ Fold  1 : int [1:3] 8 20 23
  ..$ Fold  2 : int [1:3] 9 11 21
  ..$ Fold  3 : int [1:3] 6 17 27
  ..$ Fold  4 : int [1:3] 4 13 29
  ..$ Fold  5 : int [1:3] 3 12 22
  ..$ Fold  6 : int [1:3] 5 16 30
  ..$ Fold  7 : int [1:3] 2 19 28
  ..$ Fold  8 : int [1:3] 1 18 26
  ..$ Fold  9 : int [1:3] 10 14 25
  ..$ Fold  10: int [1:3] 7 15 24
 $ Run  3:List of 10
  ..$ Fold  1 : int [1:3] 4 11 21
  ..$ Fold  2 : int [1:3] 6 20 23
  ..$ Fold  3 : int [1:3] 7 18 29
  ..$ Fold  4 : int [1:3] 1 14 22
  ..$ Fold  5 : int [1:3] 3 12 25
  ..$ Fold  6 : int [1:3] 10 16 26
  ..$ Fold  7 : int [1:3] 8 13 28
  ..$ 

<a id="2"></a>
# Classify Function

Classify function is obtained from lecture notebooks. Function takes 4 different parameters, distance matrix, class information, test indeces and k parameter.

In [13]:
nn_classify_cv=function(dist_matrix,train_class,test_indices,k){
    
    test_distances_to_train=dist_matrix[test_indices,]
    test_distances_to_train=test_distances_to_train[,-test_indices]
    train_class=train_class[-test_indices]

    ordered_indices=apply(test_distances_to_train,1,order)
    if(k==1){
        nearest_class=as.numeric(train_class[as.numeric(ordered_indices[1,])])
        nearest_class=data.table(id=test_indices,nearest_class)
    } else {
        nearest_class=apply(ordered_indices[1:k,],2,function(x) {train_class[x]})
        nearest_class=data.table(id=test_indices,t(nearest_class))
    }
    
    long_nn_class=melt(nearest_class,'id')

    class_counts=long_nn_class[,.N,list(id,value)]
    class_counts[,predicted_prob:=N/k]
    wide_class_prob_predictions=dcast(class_counts,id~value,value.var='predicted_prob')
    wide_class_prob_predictions[is.na(wide_class_prob_predictions)]=0
    class_predictions=class_counts[,list(predicted=value[which.max(N)]),by=list(id)]
    
    
    return(list(prediction=class_predictions,prob_estimates=wide_class_prob_predictions))
    
}

<a id="3"></a>
# Representations

There are 3 major representations, raw dataset, difference taken dataset, and paa representation. For difference taken dataset 1 and 2 will be used in shift operator. At the beginning, an example code will be given to show obtained dataframes in difference datasets. For paa dataset 4 and 8 will be used in segment length parameter. At the beginning, an example code will be given to show obtained dataframes in paa datasets.

## Example Code for difference datasets

In [14]:
dt_ts_train=data.table(traindata)
dt_ts_train[,id:=1:.N]
long_train=melt(dt_ts_train,id.vars=c('id'))
long_train[,time:=as.numeric(gsub("\\D", "", variable))-1]
long_train=long_train[order(id,time)]
diff_long=copy(long_train)
diff_long[,diff_series:=value-shift(value,1),by=list(id)]
head(diff_long)

id,variable,value,time,diff_series
1,V2,0.0377646330,1,NA
1,V3,0.0002531124,2,-0.037511521
1,V4,0.0039770760,3,0.003723964
1,V5,0.0207640580,4,0.016786982
1,V6,0.0217311800,5,0.000967122
1,V7,0.1140000000,6,0.092268820


In [15]:
diff_train=dcast(diff_long[!is.na(diff_series)],id~time,value.var='diff_series')
diff_train=diff_train[,-c("id")]
head(diff_train)
diff_train=as.matrix(diff_train)

2,3,4,5,6,7,8,9,10,11,...,119,120,121,122,123,124,125,126,127,128
-0.037511521,0.003723964,0.016786982,0.000967122,0.092268820,0.300000000,0.304000000,0.17100000,0.08000000,0.03300000,...,0.015133234,-0.011377629,-0.009443586,0.025519562,0.000280744,0.014429013,0.001056311,-0.023149859,0.008264213,-0.019126568
0.020556942,-0.018455991,0.017591693,0.016956416,-0.002291931,0.048217989,0.234000000,0.23300000,0.13000000,-0.02700000,...,0.007019978,-0.025736393,0.013812700,0.019324169,-0.000888445,0.000687149,-0.022396647,-0.013977540,0.021405171,-0.007356809
-0.016179174,0.031400853,-0.023532115,0.115454676,0.335000000,0.335000000,0.188000000,0.08900000,-0.09100000,-0.32400000,...,0.023940894,-0.026179138,0.028973513,-0.040942481,0.013779004,0.002096286,-0.002912475,0.003799242,0.009724722,0.012983624
0.008470759,0.019049504,-0.004757391,0.010078628,-0.023441310,0.009001748,0.077402581,0.24800000,0.24300000,0.13100000,...,-0.015041910,0.014071350,-0.016028641,0.040140282,-0.044499591,0.018007364,-0.013385823,-0.002127067,0.027963798,0.002843902
0.025712052,-0.009823282,-0.003396833,-0.016970592,0.010976473,-0.005616872,-0.020796678,0.01289733,0.01472031,0.06691766,...,0.014433514,-0.016467389,0.025773254,-0.030666813,-0.008877612,-0.005649936,0.027381701,0.001877891,0.011752984,-0.006059050
0.019603915,-0.034057333,0.030254248,-0.014962628,0.011071374,-0.019475918,-0.001109278,0.10172625,0.31400000,0.31300000,...,-0.029374361,-0.000431908,0.033117647,-0.005569584,0.010642672,0.006239214,-0.018477157,0.002842005,-0.004238373,-0.004017664


<a id="4"></a>
## Difference Function

In [16]:
difference_obtainer=function(traindata, diff_value){
    dt_ts_train=data.table(traindata)
    dt_ts_train[,id:=1:.N]
    long_train=melt(dt_ts_train,id.vars=c('id'))
    long_train[,time:=as.numeric(gsub("\\D", "", variable))-1]
    long_train=long_train[order(id,time)]
    diff_long=copy(long_train)
    diff_long[,diff_series:=value-shift(value,diff_value),by=list(id)]#Lag value is assigned by diff_value
    head(diff_long)
    
    diff_train=dcast(diff_long[!is.na(diff_series)],id~time,value.var='diff_series')
    diff_train=diff_train[,-c("id")]
    head(diff_train)
    diff_train=as.matrix(diff_train)
    
    return(diff_train)
}

This function will be used to get a difference dataset. "_2" string will be used to mention 2 differences taken dataset.

###  1 Difference

In [17]:
diff_train=difference_obtainer(traindata,1)

### 2 Difference

In [18]:
diff_train_2=difference_obtainer(traindata,2)

## Example Code for PAA

In [19]:
segment_length=5

In [20]:
paa_results=vector("list", max(long_train$id))

In [21]:
for(i in 1:max(long_train$id)){
    current_ts=long_train[id==i,]$value
    
    paa_rep=repr_paa(current_ts, segment_length, meanC)
    current_dt=data.table(time=1:length(long_train[id==i,]$value))
    result_dt=data.table(time=c(1:(length(paa_rep)))*segment_length, values=paa_rep)
    all_dt=merge(current_dt, result_dt, by='time',all.x=T)
    all_dt[,values:=nafill(values,'nocb')]
    paa_results[[i]]=transpose(data.table(values=all_dt$values))
    
}

In [22]:
paa_train=rbindlist(paa_results)

In [23]:
paa_train

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128
0.0168980119,0.0168980119,0.0168980119,0.0168980119,0.0168980119,0.620800000,0.620800000,0.620800000,0.620800000,0.620800000,...,0.02895437,0.02895437,0.03995435,0.03995435,0.03995435,0.03995435,0.03995435,NA,NA,NA
0.0162248013,0.0162248013,0.0162248013,0.0162248013,0.0162248013,0.332956402,0.332956402,0.332956402,0.332956402,0.332956402,...,0.03060810,0.03060810,0.03576214,0.03576214,0.03576214,0.03576214,0.03576214,NA,NA,NA
0.0394310218,0.0394310218,0.0394310218,0.0394310218,0.0394310218,0.860200000,0.860200000,0.860200000,0.860200000,0.860200000,...,0.03425886,0.03425886,0.02242872,0.02242872,0.02242872,0.02242872,0.02242872,NA,NA,NA
0.0125145597,0.0125145597,0.0125145597,0.0125145597,0.0125145597,0.205038618,0.205038618,0.205038618,0.205038618,0.205038618,...,0.01669614,0.01669614,0.01538142,0.01538142,0.01538142,0.01538142,0.01538142,NA,NA,NA
0.0413032538,0.0413032538,0.0413032538,0.0413032538,0.0413032538,0.029009739,0.029009739,0.029009739,0.029009739,0.029009739,...,0.02115875,0.02115875,0.01832026,0.01832026,0.01832026,0.01832026,0.01832026,NA,NA,NA
0.0313072738,0.0313072738,0.0313072738,0.0313072738,0.0313072738,0.126103144,0.126103144,0.126103144,0.126103144,0.126103144,...,0.02361924,0.02361924,0.04757112,0.04757112,0.04757112,0.04757112,0.04757112,NA,NA,NA
0.0294146025,0.0294146025,0.0294146025,0.0294146025,0.0294146025,0.018427795,0.018427795,0.018427795,0.018427795,0.018427795,...,0.02034330,0.02034330,0.03349451,0.03349451,0.03349451,0.03349451,0.03349451,NA,NA,NA
0.0113078153,0.0113078153,0.0113078153,0.0113078153,0.0113078153,0.128904942,0.128904942,0.128904942,0.128904942,0.128904942,...,0.03616909,0.03616909,0.03875980,0.03875980,0.03875980,0.03875980,0.03875980,NA,NA,NA
0.0307588139,0.0307588139,0.0307588139,0.0307588139,0.0307588139,0.460328199,0.460328199,0.460328199,0.460328199,0.460328199,...,0.02593722,0.02593722,0.02463345,0.02463345,0.02463345,0.02463345,0.02463345,NA,NA,NA
0.0288568120,0.0288568120,0.0288568120,0.0288568120,0.0288568120,0.128333054,0.128333054,0.128333054,0.128333054,0.128333054,...,0.03006264,0.03006264,0.02712369,0.02712369,0.02712369,0.02712369,0.02712369,NA,NA,NA


<a id="5"></a>
## PAA Function

In [24]:
paa_obtainer=function(traindata,segment_length){
    dt_ts_train=data.table(traindata)
    dt_ts_train[,id:=1:.N]
    long_train=melt(dt_ts_train,id.vars=c('id'))
    long_train[,time:=as.numeric(gsub("\\D", "", variable))-1]
    long_train=long_train[order(id,time)]
    
    paa_results=vector("list", max(long_train$id))
    for(i in 1:max(long_train$id)){
        current_ts=long_train[id==i,]$value

        paa_rep=repr_paa(current_ts, segment_length, meanC)
        current_dt=data.table(time=1:length(long_train[id==i,]$value))
        result_dt=data.table(time=c(1:(length(paa_rep)))*segment_length, values=paa_rep)
        all_dt=merge(current_dt, result_dt, by='time',all.x=T)
        all_dt[,values:=nafill(values,'nocb')]
        paa_results[[i]]=transpose(data.table(values=all_dt$values))

    }
    return(rbindlist(paa_results))
}

This function will be used to get a difference dataset. "_2" string will be used to mention segment lentgh determined as 10.

### Segment Length 4

In [25]:
paa_train=paa_obtainer(traindata, 4)

### Segment Length 8

In [26]:
paa_train_2=paa_obtainer(traindata, 8)

<a id="6"></a>
# Distances

In this part, distance datasets will be calculated to have 20 different combination (5 representation * 4 different distance calculation types). In addition, obtained distance datasets will be stored in a file to skip distance calculation.

In [27]:
large_number=10000

<a id="7"></a>
## Raw Dataset 

In [28]:
dist_euc=as.matrix(dist(traindata))
diag(dist_euc)=large_number
fwrite(dist_euc,sprintf('%s%s/%s_euc_raw_dist.csv', distance_path, third_dataset, third_dataset),col.names=F)

dist_dtw=as.matrix(dtwDist(traindata))
diag(dist_dtw)=large_number
fwrite(dist_dtw,sprintf('%s%s/%s_dtw_raw_dist.csv', distance_path, third_dataset, third_dataset),col.names=F)

dist_lcss=TSDatabaseDistances(traindata,distance='lcss',epsilon=0.05)
dist_lcss=as.matrix(dist_lcss)
diag(dist_lcss)=large_number
fwrite(dist_lcss,sprintf('%s%s/%s_lcss_raw_epsilon_005.csv', distance_path, third_dataset, third_dataset),col.names=F)  

dist_erp=TSDatabaseDistances(traindata,distance='erp',g=0.5)
dist_erp=as.matrix(dist_erp)
diag(dist_erp)=large_number
fwrite(dist_erp,sprintf('%s%s/%s_erp_raw_gap_005.csv', distance_path, third_dataset, third_dataset),col.names=F)  

x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table


<a id="8"></a>
## Difference taken Datasets

### First difference dataset when shift value is 1

In [29]:
dist_euc_diff=as.matrix(dist(diff_train))
diag(dist_euc_diff)=large_number
fwrite(dist_euc_diff,sprintf('%s%s/%s_euc_diff_dist.csv', distance_path, third_dataset, third_dataset),col.names=F)

dist_dtw_diff=as.matrix(dtwDist(diff_train))
diag(dist_dtw_diff)=large_number
fwrite(dist_dtw_diff,sprintf('%s%s/%s_dtw_diff_dist.csv', distance_path, third_dataset, third_dataset),col.names=F)

dist_lcss_diff=TSDatabaseDistances(diff_train,distance='lcss',epsilon=0.05)
dist_lcss_diff=as.matrix(dist_lcss_diff)
diag(dist_lcss_diff)=large_number
fwrite(dist_lcss_diff,sprintf('%s%s/%s_lcss_diff_epsilon_005.csv', distance_path, third_dataset, third_dataset),col.names=F)  

dist_erp_diff=TSDatabaseDistances(diff_train,distance='erp',g=0.5)
dist_erp_diff=as.matrix(dist_erp_diff)
diag(dist_erp_diff)=large_number
fwrite(dist_erp_diff,sprintf('%s%s/%s_erp_diff_gap_005.csv', distance_path, third_dataset, third_dataset),col.names=F)  

x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table


### Second difference dataset when shift value is 2

In [30]:
dist_euc_diff_2=as.matrix(dist(diff_train_2))
diag(dist_euc_diff_2)=large_number
fwrite(dist_euc_diff_2,sprintf('%s%s/%s_euc_diff2_dist.csv', distance_path, third_dataset, third_dataset),col.names=F)

dist_dtw_diff_2=as.matrix(dtwDist(diff_train_2))
diag(dist_dtw_diff_2)=large_number
fwrite(dist_dtw_diff_2,sprintf('%s%s/%s_dtw_diff2_dist.csv', distance_path, third_dataset, third_dataset),col.names=F)

dist_lcss_diff_2=TSDatabaseDistances(diff_train_2,distance='lcss',epsilon=0.05)
dist_lcss_diff_2=as.matrix(dist_lcss_diff_2)
diag(dist_lcss_diff_2)=large_number
fwrite(dist_lcss_diff_2,sprintf('%s%s/%s_lcss_diff2_epsilon_005.csv', distance_path, third_dataset, third_dataset),col.names=F)  

dist_erp_diff_2=TSDatabaseDistances(diff_train_2,distance='erp',g=0.5)
dist_erp_diff_2=as.matrix(dist_erp_diff_2)
diag(dist_erp_diff_2)=large_number
fwrite(dist_erp_diff_2,sprintf('%s%s/%s_erp_diff2_gap_005.csv', distance_path, third_dataset, third_dataset),col.names=F)  

x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table


<a id="9"></a>
## PAA Datasets

### First PAA dataset when segment length value is 4

In [31]:
dist_euc_paa=as.matrix(dist(paa_train))
diag(dist_euc_paa)=large_number
fwrite(dist_euc_paa,sprintf('%s%s/%s_euc_paa_dist.csv', distance_path, third_dataset, third_dataset),col.names=F)

dist_dtw_paa=as.matrix(dtwDist(paa_train))
diag(dist_dtw_paa)=large_number
fwrite(dist_dtw_paa,sprintf('%s%s/%s_dtw_paa_dist.csv', distance_path, third_dataset, third_dataset),col.names=F)

dist_lcss_paa=TSDatabaseDistances(paa_train,distance='lcss',epsilon=0.05)
dist_lcss_paa=as.matrix(dist_lcss_paa)
diag(dist_lcss_paa)=large_number
fwrite(dist_lcss_paa,sprintf('%s%s/%s_lcss_paa_epsilon_005.csv', distance_path, third_dataset, third_dataset),col.names=F)  

dist_erp_paa=TSDatabaseDistances(paa_train,distance='erp',g=0.5)
dist_erp_paa=as.matrix(dist_erp_paa)
diag(dist_erp_paa)=large_number
fwrite(dist_erp_paa,sprintf('%s%s/%s_erp_paa_gap_005.csv', distance_path, third_dataset, third_dataset),col.names=F)  

x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table


### Second PAA dataset when segment length value is 8

In [32]:
dist_euc_paa_2=as.matrix(dist(paa_train_2))
diag(dist_euc_paa_2)=large_number
fwrite(dist_euc_paa_2,sprintf('%s%s/%s_euc_paa2_dist.csv', distance_path, third_dataset, third_dataset),col.names=F)

dist_dtw_paa_2=as.matrix(dtwDist(paa_train_2))
diag(dist_dtw_paa_2)=large_number
fwrite(dist_dtw_paa_2,sprintf('%s%s/%s_dtw_paa2_dist.csv', distance_path, third_dataset, third_dataset),col.names=F)

dist_lcss_paa_2=TSDatabaseDistances(paa_train_2,distance='lcss',epsilon=0.05)
dist_lcss_paa_2=as.matrix(dist_lcss_paa_2)
diag(dist_lcss_paa_2)=large_number
fwrite(dist_lcss_paa_2,sprintf('%s%s/%s_lcss_paa2_epsilon_005.csv', distance_path, third_dataset, third_dataset),col.names=F)  

dist_erp_paa_2=TSDatabaseDistances(paa_train_2,distance='erp',g=0.5)
dist_erp_paa_2=as.matrix(dist_erp_paa_2)
diag(dist_erp_paa_2)=large_number
fwrite(dist_erp_paa_2,sprintf('%s%s/%s_erp_paa2_gap_005.csv', distance_path, third_dataset, third_dataset),col.names=F)  

x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table
x being coerced from class: matrix to data.table


### To store obtained distance datasetes in a list

In [33]:
dist_folder=sprintf('%s%s', distance_path, third_dataset)
dist_files=list.files(dist_folder, full.names=T)

In [34]:
dist_files

[1] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BME/BME_dtw_diff_dist.csv"         
 [2] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BME/BME_dtw_diff2_dist.csv"        
 [3] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BME/BME_dtw_paa_dist.csv"          
 [4] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BME/BME_dtw_paa2_dist.csv"         
 [5] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BME/BME_dtw_raw_dist.csv"          
 [6] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BME/BME_erp_diff_gap_005.csv"      
 [7] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BME/BME_erp_diff2_gap_005.csv"     
 [8] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BME/BME_erp_paa_gap_005.csv"       
 [9] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BME/BME_erp_paa2_gap_005.csv"      
[10] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BME/BME_erp_raw_gap_005.csv"       
[11] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BME/BME_euc_diff_dist.csv"         
[12] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BME/BME_euc_diff2_dist.csv"        
[13] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BME/BME_euc_paa_dist.csv"          
[14] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BME/BME_euc_paa2_dist.csv"         
[15] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BME/BME_euc_raw_dist.csv"          
[16] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BME/BME_lcss_diff_epsilon_005.csv" 
[17] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BME/BME_lcss_diff2_epsilon_005.csv"
[18] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BME/BME_lcss_paa_epsilon_005.csv"  
[19] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BME/BME_lcss_paa2_epsilon_005.csv" 
[20] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BME/BME_lcss_raw_epsilon_005.csv"

<a id="10"></a>
# Main Model

In [35]:
k_levels=c(1,3,5)
approach_file=list.files(dist_folder)

In [36]:
result=vector('list',length(dist_files)*nof_rep*n_fold*length(k_levels))

In [37]:
iter=1
for(m in 1:length(dist_files)){ #
    print(dist_files[m])
    dist_mat=as.matrix(fread(dist_files[m],header=FALSE))
    for(i in 1:nof_rep){
        this_fold=cv_indices[[i]]
        for(j in 1:n_fold){
            test_indices=this_fold[[j]]
            for(k in 1:length(k_levels)){
                current_k=k_levels[k]
                current_fold=nn_classify_cv(dist_mat,trainclass,test_indices,k=current_k)
                accuracy=sum(trainclass[test_indices]==current_fold$prediction$predicted)/length(test_indices)
                tmp=data.table(approach=approach_file[m],repid=i,foldid=j,
                               k=current_k,acc=accuracy)
                result[[iter]]=tmp
                iter=iter+1
            }
            
        }
    
    }   
    
}

[1] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BME/BME_dtw_diff_dist.csv"
[1] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BME/BME_dtw_diff2_dist.csv"
[1] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BME/BME_dtw_paa_dist.csv"
[1] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BME/BME_dtw_paa2_dist.csv"
[1] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BME/BME_dtw_raw_dist.csv"
[1] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BME/BME_erp_diff_gap_005.csv"
[1] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BME/BME_erp_diff2_gap_005.csv"
[1] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BME/BME_erp_paa_gap_005.csv"
[1] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BME/BME_erp_paa2_gap_005.csv"
[1] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BME/BME_erp_raw_gap_005.csv"
[1] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BME/BME_euc_diff_dist.csv"
[1] "C:/Users/bahad/GitHub/IE48B/Homework3/Distances/BME/BME_euc_diff2_dist.csv"
[1] "C:/Users/bahad/GitHub

<a id="11"></a>
# Result of Models

In [38]:
dataframe_result=rbindlist(result)
head(dataframe_result)

approach,repid,foldid,k,acc
BME_dtw_diff_dist.csv,1,1,1,1
BME_dtw_diff_dist.csv,1,1,3,1
BME_dtw_diff_dist.csv,1,1,5,1
BME_dtw_diff_dist.csv,1,2,1,1
BME_dtw_diff_dist.csv,1,2,3,1
BME_dtw_diff_dist.csv,1,2,5,1


In this dataset, result of each fold exists in this dataframe. repid and foldid represent which repetition and fold respectively. 

### Accumulated Datasets

In [39]:
acc_res=dataframe_result[,list(avg_acc=mean(acc),sdev_acc=sd(acc), repid=max(repid), foldid=max(foldid), 
                                   result_count=.N),by=list(approach,k)]
acc_res_ordered=acc_res[order(avg_acc,decreasing = TRUE)]

Accumulated dataset are ordered by avg_acc value. 

In [40]:
acc_res_ordered

approach,k,avg_acc,sdev_acc,repid,foldid,result_count
BME_dtw_diff2_dist.csv,1,1.00000000,0.00000000,5,10,50
BME_dtw_diff2_dist.csv,3,1.00000000,0.00000000,5,10,50
BME_dtw_diff2_dist.csv,5,0.96666667,0.10101525,5,10,50
BME_dtw_diff_dist.csv,1,0.94000000,0.12936264,5,10,50
BME_dtw_diff_dist.csv,5,0.94000000,0.12936264,5,10,50
BME_dtw_diff_dist.csv,3,0.93333333,0.13468701,5,10,50
BME_euc_paa_dist.csv,1,0.90000000,0.15430335,5,10,50
BME_euc_paa2_dist.csv,1,0.90000000,0.15430335,5,10,50
BME_euc_raw_dist.csv,1,0.87333333,0.16343812,5,10,50
BME_dtw_paa2_dist.csv,1,0.87333333,0.16343812,5,10,50


In [41]:
# require(ggplot2)
# ggplot(dataframe_result,aes(x=paste0(approach,'with K=',k), y=acc)) +
#         geom_boxplot()+
#         labs(title="Boxplot of Models")+
#         xlab("Model Types")+
#         coord_flip()

## Best Result and Parameter Combination

In [54]:
acc_res_ordered[1]

approach,k,avg_acc,sdev_acc,repid,foldid,result_count
BME_dtw_diff2_dist.csv,1,1,0,5,10,50


Result is obtained by considering mean and standard deviation of the accuracy over the test folds. There are 2 different models obtaining the full accuracy. The only difference in these models are the k values. K is selected as 1 to have simple model.

### Best Parameter Combination

- Representation: 2 Difference Taken Data
- Distance: Dynamic Time Warping with epsilon 0.05
- K value of NN: 1

**Average Accuracy: 1.00 based on 5 repeats and 10-fold cross validation**

<a id="12"></a>
# Test Performance

Best performance is obtained when representation, distance, and k values are difference=1, Euclidean, and K=1 respectively.

In [42]:
traindata=as.matrix(fread(sprintf('%s%s/%s_TRAIN.txt',dataset_path, third_dataset,third_dataset)))
testdata=as.matrix(fread(sprintf('%s%s/%s_TEST.txt',dataset_path, third_dataset,third_dataset)))

To get test performance, train and test datasets will be used. These 2 datasets will be bind and test indices are selected as test dataset indices.

In [43]:
all_dt=rbind(traindata, testdata)

In [44]:
allclass=all_dt[,1] 
all_dt=all_dt[,2:ncol(all_dt)]

In [45]:
test_indices_last=(nrow(all_dt)+1-nrow(testdata)):nrow(all_dt)

### Parameters

In [46]:
test_indices_last

[1]  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48
 [19]  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66
 [37]  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84
 [55]  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101 102
 [73] 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120
 [91] 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138
[109] 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156
[127] 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174
[145] 175 176 177 178 179 180

In [47]:
last_k=1

## Representation and Distance Calculation

Representation and distance types are selected by looking the best parameter combination obtained in train dataset.

In [48]:
diff_test_2=difference_obtainer(all_dt,2)

In [49]:
dist_dtw_diff_2_test=as.matrix(dtwDist(diff_test_2))
diag(dist_dtw_diff_2_test)=large_number

## Result of Test Dataset

In [50]:
last_result=nn_classify_cv(dist_dtw_diff_2_test,allclass,test_indices_last,k=last_k)
accuracy=sum(allclass[test_indices_last]==last_result$prediction$predicted)/length(test_indices_last)
final_res=data.table(approach="BME_dtw_diff2_dist_Test.csv", k=last_k, acc=accuracy)

### Test Result

In [51]:
final_res

approach,k,acc
BME_dtw_diff2_dist_Test.csv,1,0.98


### Train Result

In [52]:
acc_res_ordered[1][,c("approach", "k", "avg_acc")]

approach,k,avg_acc
BME_dtw_diff2_dist.csv,1,1


<a id="13"></a>
# Comments

Obtained results will be analyzed in the result comparison notebook.